# Multi-Agent

              +----------------+
              | Manager agent  |
              +----------------+
                       |
        _______________|______________
       |                              |
Code Interpreter            +------------------+
    tool                    | Web Search agent |
                            +------------------+
                               |            |
                        Web Search tool     |
                                   Visit webpage tool

In [1]:
! pip install markdownify duckduckgo-search smolagents --upgrade -q

In [5]:
from huggingface_hub import login

In [7]:
model_id = "Qwen/Qwen2.5-Coder-32B-Instruct"

## Web search tool

In [16]:
import re
import requests
from markdownify import markdownify
from requests.exceptions import RequestException
from smolagents import tool
import pandas as pd

@tool
def visit_webpage(url: str) -> str:
    """Visits a webpage at the given URL and returns its content as a markdown string.

    Args:
        url: The URL of the webpage to visit.

    Returns:
        The content of the webpage converted to Markdown, or an error message if the request fails.
    """
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Convert the HTML content to Markdown
        markdown_content = markdownify(response.text).strip()

        # Remove multiple line breaks
        markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)
        # Remove markdown headers
        markdown_content = re.sub(r"^#+ ", "", markdown_content) 

        return markdown_content

    except RequestException as e:
        return f"Error fetching the webpage: {str(e)}"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

In [ ]:
print(visit_webpage("https://en.wikipedia.org/wiki/Hugging_Face")[:500])

# Multi-Agent System


In [17]:
from smolagents import (
    CodeAgent,
    ToolCallingAgent,
    HfApiModel,
    DuckDuckGoSearchTool,
    LiteLLMModel,
)

model = HfApiModel(model_id)

web_agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool(), visit_webpage],
    model=model,
    max_steps=1,
    name="web_search_agent",
    description="Runs web searches for you.",
)

In [18]:
manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[web_agent],
    additional_authorized_imports=["time", "numpy", "pandas", "json"],
)

In [ ]:
wine_data = manager_agent.run("Make a comparative table of all wine cépages for red wine in France with the following information: name, color, grape variety, region, price, and rating.")


In [ ]:
## Read a json cepages.json
with open('cepages.json', 'r') as f:
    cepages = json.load(f)

wine_table = pd.DataFrame(cepages)

# Display the table
wine_table.head()

# Save to CSV
wine_table.to_csv("wine_comparative_table.csv", index=False)

print("Table saved as 'wine_comparative_table.csv'")

# Use the web_agent to get the data
wine_data = web_agent.run("Make a comparative table of all wine cépages for red wine in France with the following information: name.")

# Convert the wine_data text to a DataFrame
from io import StringIO
import pandas as pd
wine_table = pd.read_csv(StringIO(str(wine_data)), sep="|")



In [ ]:
# Save to CSV
wine_table.to_csv("wine_comparative_table.csv", index=False)

print("Table saved as 'wine_comparative_table.csv'")